In [5]:
import resources as r

import ipywidgets as widgets
from IPython.display import HTML, display

display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
    .widget-inline-hbox { min-width: 40ex !important; }
</style>'''))

layout = widgets.Layout(width='50%')

fluorophores, solvents, methods = r.fluorophores_solvents_methods()
basisOptions = r.Basis
methodOptions = r.Methods

def submitORCA(memory:int, cores:int, partner:bool, jobType:r.Jobs, stateList:list[r.States], grid:r.Grids, basisList:list[r.Basis], methodList:list[r.Methods], 
               fluorophoreList:list[r.Fluorophores], uses0:bool, submit:bool, resubmit:bool, resetStatus:bool) -> None:
    with r.statusLoad('progress_gas') as df_gas:
        with r.statusLoad('progress_s1') as df_s1:
            with r.statusLoad('progress_s0') as df_s0:
                with r.statusLoad('progress_s2') as df_s2:
                    dfDict = {r.States.s0: df_s0, r.States.s1: df_s1, r.States.s2: df_s2}
                    with r.monarchHandler() as mon:
                        for fluorophore in fluorophoreList:
                            for method in methodList:
                                for basis in basisList:
                                    for state in stateList:
                                        status = df_gas.at[(fluorophore, method), state]
                                        if status in [r.Status.SMD.failed, None] or resubmit:
                                            xyzfile = ''
                                            mofile = ''
                                            catxyzpath = ''
                                            scfstring = ''
                                            kdiis = False
                                            soscf = True
                                            notrah = False

                                            status_s0 = dfDict[r.States.s0].at[(fluorophore, method), r.Solvents.tol]

                                            crest_refJob = r.Job(r.Software.crest, fluorophore, r.Solvents.tol, r.Methods.crest, r.Basis.none, r.PCM.alpb, r.PCM.Eq.none, r.States.s0, r.Jobs.crest)
                                            wb_refJob = r.Job(r.Software.orca, fluorophore, r.Solvents.tol, r.Methods.wb97xd, r.Basis.augccpvdz, r.PCM.smd, r.PCM.Eq.eq, r.States.s0, r.Jobs.opt)
                                            if status_s0 in [r.Status.Crest.finished, r.Status.SMD.finished, None] or resubmit == True:
                                                if status_s0 == r.Status.SMD.finished:
                                                    refJob = wb_refJob
                                                else:
                                                    print('ground state opt not finished, using CREST geom.')
                                                    refJob = crest_refJob
                                            else:
                                                refJob = crest_refJob
                                                
                                            job = r.Job(r.Software.orca, fluorophore, r.Solvents.gas, method, basis, r.PCM.none, r.PCM.Eq.none, state, jobType, grid=grid,
                                                        mopath=mofile, xyzpath=xyzfile, catxyzpath=refJob.xyzfile, kdiis=kdiis, soscf=soscf, notrah=notrah, scfstring=scfstring,
                                                        submit=submit, submitFlags=f'-NSs', partner=partner, procs=cores, mem=memory)

                                            fileName = mon.buildJob(job)

                                            print(job.name)
                                            if submit == True:
                                                if resetStatus:
                                                    df_gas.at[(fluorophore, method), state] = r.Status.SMD.queued
                                        elif status in [r.Status.SMD.finished, r.Status.SMD.failed, r.Status.SMD.queued, r.Status.SMD.running]:
                                            print(f'{fluorophore.name} Orca job already submitted, use "resubmit" to overwrite the job file')

    print('Done!')

widgets.interactive(submitORCA, {'manual' : True, 'manual_name' : 'Build ORCA Job'},
                     jobType=widgets.Dropdown(options=r.Jobs, rows=6, description='Job Type'),
                     fluorophoreList=widgets.SelectMultiple(options=fluorophores, rows=len(fluorophores), description='Fluorophore'),
                     basisList=widgets.SelectMultiple(options=r.Basis, rows=15, description='Basis'),
                     grid=widgets.Dropdown(options=r.Grids, rows=len(r.Grids), description='Grids', value=r.Grids.g75_302),
                     methodList=widgets.SelectMultiple(options=r.Methods, rows=len(r.Methods), description='Method'),
                     stateList=widgets.SelectMultiple(options=r.States, rows=4, description='States'),
                     partner = widgets.Checkbox(value=True, description='Partner'),
                     uses0 = widgets.Checkbox(value=False, description='Use S0 opt for higher root starting geometry', layout=layout),
                     submit = widgets.Checkbox(value=False, description='Submit'),
                     resubmit = widgets.Checkbox(value=False, description='ReSubmit'),
                     resetStatus = widgets.Checkbox(value=False, description='Reset Status'),
                     memory = widgets.IntText(value=64, description='Memory'),
                     cores = widgets.IntText(value=16, description='Cores'),
                     )

interactive(children=(IntText(value=64, description='Memory'), IntText(value=16, description='Cores'), Checkbo…